# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [1]:
import pandas as pd
import numpy as np
import folium

In [2]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv('../../DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [3]:
# install 
!pip install scikit-optimize

### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [4]:
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.to_numpy()[:10000,:], y.to_numpy()[:10000]) 

In [5]:
#-----------RandomForestRegressor-------------#
import warnings
warnings.filterwarnings('ignore', message='The objective has been evaluated at this point before.')
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor

rfr_opt = BayesSearchCV( 
         RandomForestRegressor(),
         {
             'n_estimators':[50, 150, 200, 250, 300, 350],
             'max_depth':[2, 4, 6, 8, 10, 12, 14]
         },
         n_iter=32,
         random_state=0,
         n_jobs=-1,
         cv=3
     )

rfr_mod = rfr_opt.fit(X_train, y_train)
print("Score:", rfr_mod.score(X_test, y_test))
print("Best Score:",rfr_mod.best_score_)
print("Best Params:",rfr_mod.best_params_)

Score: 0.8353435869712948
Best Score: 0.7755582923512498
Best Params: OrderedDict([('max_depth', 10), ('n_estimators', 250)])


In [6]:
#-----------SVR-------------#

from sklearn.svm import SVR
import warnings
warnings.filterwarnings('ignore', message='The objective has been evaluated at this point before.')

svr_opt = BayesSearchCV(
    SVR(),
    {
        'C': [0.1, 0.2, 0.3, 0.5, 1.0],
        'gamma': [0.1, 0.01, 0.001, 0.0001, 1.0],
        'kernel': ['rbf'],
    },
    n_iter=32,
    random_state=0,
    n_jobs=-1,
    pre_dispatch='n_jobs'
)


svr_mod = svr_opt.fit(X_train, y_train)
print("Score:", svr_mod.score(X_test, y_test))
print("Best Score:",svr_mod.best_score_)
print("Best Params:",svr_mod.best_params_)

Score: 0.7781697737235646
Best Score: 0.733099523440998
Best Params: OrderedDict([('C', 1.0), ('gamma', 0.001), ('kernel', 'rbf')])


In [7]:
#-----------Linear Regression-------------#

from sklearn.linear_model import LinearRegression
lin = LinearRegression()
lin.fit(X_train,y_train)
print(lin.score(X_test, y_test))

0.7401300906142674


In [8]:
#-------------Check best params-------------#

#rfr
rnd_best = RandomForestRegressor(
    n_estimators=150,
    max_depth=8)
rnd_optimum = rnd_best.fit(X_train, y_train)
print(rnd_optimum.score(X_test, y_test))

0.8320936826414214


In [9]:
rnd_optimum = rnd_best.fit(X_train, y_train)
print(rnd_optimum)
print(rnd_optimum.score(X_test, y_test))

RandomForestRegressor(max_depth=8, n_estimators=150)
0.8315736727185364


In [10]:
svr_best=SVR(C=1.0,
        gamma= 0.001,
        kernel= 'rbf')
svr_optimum = svr_best.fit(X_train, y_train)
print(svr_optimum.score(X_test, y_test))

0.7781697737235646
